In [42]:
import pandas as pd
import numpy as np
import re

In [43]:
inters = pd.read_csv('~/data/train_ratings.csv')
users = pd.read_csv('~/data/users.csv')
books = pd.read_csv('~/data/books.csv')

In [44]:
# Users

In [45]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   68092 non-null  int64  
 1   location  68092 non-null  object 
 2   age       40259 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [46]:
# location

In [47]:
states_abbreviations = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

states_abbreviations_lowercase = {k.lower(): v.lower() for k,v in states_abbreviations.items()}

def preprocess_location(location):
    country = re.sub("\s+", '', location.split(',')[-1])
    if country == 'usa':
        state = re.sub("\s+", '', location.split(',')[-2])
        
        if state in states_abbreviations_lowercase:
            state = states_abbreviations_lowercase[state]
            
        return country + '-' + state
    else:
        return country

In [48]:
users['location_country'] = users['location'].apply(lambda x: preprocess_location(x)).replace(['theworldtomorrow', 'unknown', 'n/a', ''], np.NaN)

In [49]:
users['location_first'] = users['location'].apply(lambda x: re.sub("\s+", '', x.split(',')[0])).replace(['whoknows', 'unknown', 'n/a', ''], np.NaN)

In [50]:
users.head(20)

,user_id,location,age,location_country,location_first
0,8,"timmins, ontario, canada",NaN,canada,timmins
1,11400,"ottawa, ontario, canada",49.0,canada,ottawa
2,11676,"n/a, n/a, n/a",NaN,NaN,NaN
3,67544,"toronto, ontario, canada",30.0,canada,toronto
4,85526,"victoria, british columbia, canada",36.0,canada,victoria
5,96054,"ottawa, ontario, canada",29.0,canada,ottawa
6,116866,"ottawa, ,",NaN,NaN,ottawa
7,123629,"kingston, ontario, canada",NaN,canada,kingston
8,177458,"ottawa, ontario, canada",29.0,canada,ottawa
9,200273,"comber, ontario, canada",NaN,canada,comber


In [51]:
def substitute_location(row):
    if (type(row['location_country']) == str) | (type(row['location_first']) != str):
        return row
    
    sub_row = users[(users['location_first'] == row['location_first']) & (users['location_country'].notna())][['location_country', 'location_first']]
    if sub_row.shape[0] == 0:
        return row
    else:
        row['location_country'] = sub_row.groupby(sub_row.columns.tolist(),as_index=False).size()['location_country'][0]
        return row

In [52]:
users = users.apply(lambda x: substitute_location(x), axis = 1)

In [53]:
users = users.drop(['location', 'location_first'], axis=1)

In [54]:
location_threshold = 100

country_freq = users['location_country'].value_counts()
low_freq_country = country_freq[country_freq < location_threshold].index.tolist()

def combine_low_countries(country):
    if country in low_freq_country:
        return 'others'
    else:
        return country

users['location_country'] = users['location_country'].apply(lambda x: combine_low_countries(x))

In [55]:
users.head()

,user_id,age,location_country
0,8,NaN,canada
1,11400,49.0,canada
2,11676,NaN,NaN
3,67544,30.0,canada
4,85526,36.0,canada


In [56]:
users['location_country'] = users['location_country'].fillna(users['location_country'].value_counts().index.tolist()[0])

In [57]:
users.head()

,user_id,age,location_country
0,8,NaN,canada
1,11400,49.0,canada
2,11676,NaN,usa-california
3,67544,30.0,canada
4,85526,36.0,canada


In [58]:
# Books

In [59]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [60]:
# book_title & summary

In [61]:
books['summary'] = books['summary'].fillna('None')

In [62]:
def text_preprocessing(text):
    text = re.sub("[.,\'\"''""!?]", "", text)
    text = re.sub("[^0-9a-zA-Z\\s]", " ", text)
    text = re.sub("\s+", " ", text)
    text = text.lower()
    return text

In [63]:
books['book_title'] = books['book_title'].apply(lambda x: text_preprocessing(x))
books['summary'] = books['summary'].apply(lambda x: text_preprocessing(x))

In [64]:
# publisher

In [65]:
books['publisher'] = books['publisher'].apply(lambda x: x.lower())

In [66]:
# book_author

In [67]:
def author_preprocessing(author):
    author = re.sub("[.]", " ", author)
    author = re.sub("\s+", " ", author)
    author_parts = author.split()
    new_name = []
    for part in author_parts[:-1]:
        if part != '':
            new_name.append(part[0].lower())
    new_name.append(author_parts[-1].lower())
    return ' '.join(new_name)

In [68]:
books['book_author'] = books['book_author'].apply(lambda x: author_preprocessing(x))

In [69]:
# language

In [70]:
books['language'].value_counts()

en       78823
de        1282
es        1017
fr         883
it         123
nl          67
pt          47
da          37
ca          23
ms          10
no           6
zh-CN        3
ja           3
gl           3
ru           3
la           3
el           1
th           1
ro           1
fa           1
eo           1
gd           1
ga           1
vi           1
zh-TW        1
ko           1
Name: language, dtype: int64

In [71]:
language_threshold = 10

language_freq = books['language'].value_counts()
low_freq_language = language_freq[language_freq < language_threshold].index.tolist()

def combine_low_languages(language):
    if language in low_freq_language:
        return 'others'
    else:
        return language

books['language'] = books['language'].apply(lambda x: combine_low_languages(x))

In [72]:
books['language'] = books['language'].fillna(books['language'].value_counts().index.tolist()[0])

In [73]:
books['language'].value_counts()

en        146050
de          1282
es          1017
fr           883
it           123
nl            67
pt            47
da            37
others        31
ca            23
ms            10
Name: language, dtype: int64

In [74]:
# category

In [75]:
books['category'] = books['category'].apply(lambda x: x.lower() if type(x) == str else x)

In [76]:
books['category'].value_counts().tail(20)

['brenner, simon (fictitious character)']                 1
['computer software industry']                            1
['novela española - siglo xx']                            1
['women television producers and directors']              1
['vocational guidance']                                   1
['star trek (television show)']                           1
['recovered memory']                                      1
['eschatology']                                           1
['dogsledding']                                           1
['gone with the wind (motion picture : 1939)']            1
['aeronautics, military']                                 1
['c# (computer program language)']                        1
['disguise']                                              1
['country lawyers']                                       1
['humorous stories, brazilian']                           1
['coasts']                                                1
['unix (computer file)']                

In [77]:
# Inters

In [78]:
train = inters.merge(users, on='user_id', how='left').merge(books.drop(['img_url', 'img_path'], axis=1), on='isbn', how='left')

In [79]:
train.head()

,user_id,isbn,rating,age,location_country,book_title,book_author,year_of_publication,publisher,language,category,summary
0,8,0002005018,4,NaN,canada,clara callan,r b wright,2001.0,harperflamingo canada,en,['actresses'],in a small town in canada clara callan relucta...
1,67544,0002005018,7,30.0,canada,clara callan,r b wright,2001.0,harperflamingo canada,en,['actresses'],in a small town in canada clara callan relucta...
2,123629,0002005018,8,NaN,canada,clara callan,r b wright,2001.0,harperflamingo canada,en,['actresses'],in a small town in canada clara callan relucta...
3,200273,0002005018,8,NaN,canada,clara callan,r b wright,2001.0,harperflamingo canada,en,['actresses'],in a small town in canada clara callan relucta...
4,210926,0002005018,9,NaN,canada,clara callan,r b wright,2001.0,harperflamingo canada,en,['actresses'],in a small town in canada clara callan relucta...


In [80]:
books.to_csv('books.csv')
users.to_csv('users.csv')
inters.to_csv('inters.csv')